In [1]:
%%writefile mapper_group.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

def to_hash(word):
    return ''.join(sorted(word))

def is_word(word):
    return (len(word) > 1 and word.isalpha())

stop_hashes = set()
with open('stop_words_en.txt') as f:
    stop_hashes = set(map(lambda x: to_hash(x), f.read().split()))
    
for line in sys.stdin:
    try:
        article_id, content = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
        
    words = re.split('\W*\s+\W*', content, flags=re.UNICODE)
    
    for word in words:
#         word = re.sub("^\W+|\W+$", "", word, flags=re.UNICODE)
        word = word.lower()
        if is_word(word):
            group_hash = to_hash(word)
            
            if not group_hash in stop_hashes:
                print('%s\t%d\t%s' % (group_hash, 1, word))

Overwriting mapper_group.py


In [2]:
%%writefile reducer_group.py

import sys

reload(sys)
sys.setdefaultencoding('utf-8')

cur_group_hash = None
cur_group_occurance = 0
cur_group_set = set()

for line in sys.stdin:
    try:
        group_hash, group_occurance, words = line.strip().split('\t', 2)
        group_occurance = int(group_occurance)
        group_set = set(words.split(','))
    except ValueError as e:
        continue
    
    if group_hash != cur_group_hash:
        if cur_group_hash:
            print('%s\t%d\t%s' % (cur_group_hash, cur_group_occurance, ','.join(sorted(cur_group_set))))
            
        cur_group_hash = group_hash
        cur_group_occurance = 0
        cur_group_set = set()
        
    cur_group_occurance += group_occurance
    cur_group_set.update(group_set)
    
if cur_group_hash:
    print('%s\t%d\t%s' % (cur_group_hash, cur_group_occurance, ','.join(sorted(cur_group_set))))

Overwriting reducer_group.py


In [3]:
%%writefile mapper_sort.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        group_hash, group_occurance, words = line.strip().split('\t', 2)
        group_occurance = int(group_occurance)
    except ValueError as e:
        continue
    
    group_capacity = len(words.split(','))
    
    if group_capacity > 1:
        print('%d\t%d\t%s' % (group_occurance, group_capacity, words))

Overwriting mapper_sort.py


In [4]:
%%bash

OUT_DIR="word_groups"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming word groups" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper_group.py,reducer_group.py,/datasets/stop_words_en.txt \
    -mapper "python mapper_group.py" \
    -combiner "python reducer_group.py" \
    -reducer "python reducer_group.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
    
OUT_DIR1="word_groups_sort"$(date +"%s%6N")
NUM_REDUCERS1=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming word groups sort" \
    -D mapreduce.job.reduces=${NUM_REDUCERS1} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options=-k1,1nr \
    -D stream.num.map.output.key.fields=3 \
    -files mapper_sort.py \
    -mapper "python mapper_sort.py" \
    -reducer org.apache.hadoop.mapred.lib.IdentityReducer \
    -input ${OUT_DIR}/part-* \
    -output ${OUT_DIR1} > /dev/null
    
hdfs dfs -cat ${OUT_DIR1}/part-00000 | grep english | head -1

7820	5	english,helsing,hesling,shengli,shingle	


rm: `word_groups1544630054215319': No such file or directory
18/12/12 15:54:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/12 15:54:18 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/12 15:54:19 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/12 15:54:19 INFO mapreduce.JobSubmitter: number of splits:2
18/12/12 15:54:20 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1544629076220_0013
18/12/12 15:54:20 INFO impl.YarnClientImpl: Submitted application application_1544629076220_0013
18/12/12 15:54:20 INFO mapreduce.Job: The url to track the job: http://c6015b7593c2:8088/proxy/application_1544629076220_0013/
18/12/12 15:54:20 INFO mapreduce.Job: Running job: job_1544629076220_0013
18/12/12 15:54:26 INFO mapreduce.Job: Job job_1544629076220_0013 running in uber mode : false
18/12/12 15:54:26 INFO mapreduce.Job:  map 0% reduce 0%
18/12/12 15:54:43 INFO mapreduce.Job:  map 40% reduce 0%
18/12/12 15:54:49 I